In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import re
import string

In [2]:
dailyreview = pd.read_excel("C:\\Users\\rache\\OneDrive\\Desktop\\Practicum Project\\Daily look reviews.xlsx")
dailyreview

,Rating out of 5,Customer_Reviews
0,5,I love the surprises my stylist sends me in ea...
1,1,DON’T DO IT!!! They give you like 10 items in ...
2,5,I love that I have a personal shopper at my fi...
3,5,I love that you send so many pieces! Great var...
4,5,"My stylist, Annie has been terrific about comm..."
...,...,...
1617,5,I love their options and custom clothing sugge...
1618,5,So much better than StitchFix. My stylist actu...
1619,5,I'm on my 10th box now. I hate shopping and I ...
1620,1,"In early June, I ordered a quarterly subscript..."


In [5]:
dailyreviewdp = dailyreview.dropna()
len(dailyreviewdp)

1547

In [6]:
dailyreviewdp = dailyreviewdp.rename(columns={'Customer_Reviews': 'reviews','Rating out of 5' : 'ratings'})
dailyreviewdp.head()

,ratings,reviews
0,5,I love the surprises my stylist sends me in ea...
1,1,DON’T DO IT!!! They give you like 10 items in ...
2,5,I love that I have a personal shopper at my fi...
3,5,I love that you send so many pieces! Great var...
4,5,"My stylist, Annie has been terrific about comm..."


In [7]:
def create_sentiment(ratings):
    if ratings==1 or ratings==2:
        return -1 # negative sentiment
    elif ratings==4 or ratings==5:
        return 1 # positive sentiment
    else:
        return 0 # neutral sentiment

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
dailyreviewdp['sentiment'] = dailyreviewdp['ratings'].apply(create_sentiment)
dailyreviewdp.head()

,ratings,reviews,sentiment
0,5,I love the surprises my stylist sends me in ea...,1
1,1,DON’T DO IT!!! They give you like 10 items in ...,-1
2,5,I love that I have a personal shopper at my fi...,1
3,5,I love that you send so many pieces! Great var...,1
4,5,"My stylist, Annie has been terrific about comm...",1


In [11]:
#change to lowercase
dailyreviewdp['reviews'] = dailyreviewdp['reviews'].str.lower()

In [12]:
punct = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{}~'   # `|` is not present here
transtab = str.maketrans(dict.fromkeys(punct, ''))
dailyreviewdp['reviews'] = '|'.join(dailyreviewdp['reviews'].tolist()).translate(transtab).split('|')

In [16]:
#remove emojis
def remove_emoji(reviews):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', reviews)

In [17]:
#removing emojis
dailyreviewdp['reviews'] = dailyreviewdp['reviews'].apply(remove_emoji)
dailyreviewdp['reviews'][744]

'let me just say that the clothes that i get the most compliments on are the ones that were picked for me by dailylook i think thats about the best recommendation there is'

In [18]:
#Convert textual data into a numeric representation
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None, 
                        lowercase=False,
                        preprocessor=None)

X = tfidf.fit_transform(dailyreviewdp['reviews'])